In [65]:

import os
os.chdir('c:\\Users\\ma11115\\OneDrive - Imperial College London\\github_folder\\STK_SEARCH\\')


from stk_search import Searched_space
import importlib
importlib.reload(Searched_space)
import pickle
import pymongo
import pandas as pd
import numpy as np
from stk_search import Database_utils


In [4]:
#%% 
# Load the searched space
df_path = 'data/output/Full_datatset/df_total_new2023_08_20.csv'
df_precursors_path = 'Data/output/Prescursor_data/calculation_data_precursor_310823_clean.pkl'
#df_total, df_precursors = Database_utils.load_data_from_file(df_path, df_precursors_path)
importlib.reload(Database_utils)
df_total, df_precursors = Database_utils.load_data_database(df_precursors_path)

In [10]:
df_path = 'data/output/Full_datatset/df_total_2023_10_30.csv'
df_total.to_csv(df_path, index=False)

In [67]:
#%%
SP = Searched_space.Searched_Space(number_of_fragments=6,df=df_precursors,features_frag=df_precursors.columns[0:1],generation_type='conditional')
print(SP.features_frag)
SP.generate_interactive_condition_V2(df_total=df_total)

Index(['InChIKey'], dtype='object')


1


[[], [], [], [], [], []]
[{'number of elements': 922417564483584, 'syntax': [0, 1, 2, 3, 4, 5], 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': 0, 'number of elements evaluated': 0}, {'number of elements': 97344, 'syntax': (0, 1, 0, 1, 0, 1), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '20.0', 'number of elements evaluated': '5.00e+01'}]
2


[[], [], [], [], [], []]
[{'number of elements': 922417564483584, 'syntax': [0, 1, 2, 3, 4, 5], 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': 0, 'number of elements evaluated': 0}, {'number of elements': 97344, 'syntax': (0, 1, 0, 1, 0, 1), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '20.0', 'number of elements evaluated': '5.00e+01'}, {'number of elements': 30371328, 'syntax': (0, 1, 2, 0, 1, 2), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '15.048543689320388', 'number of elements evaluated': '2.06e+02'}]
3


[[], [], [], [], [], []]
[{'number of elements': 922417564483584, 'syntax': [0, 1, 2, 3, 4, 5], 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': 0, 'number of elements evaluated': 0}, {'number of elements': 97344, 'syntax': (0, 1, 0, 1, 0, 1), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '20.0', 'number of elements evaluated': '5.00e+01'}, {'number of elements': 30371328, 'syntax': (0, 1, 2, 0, 1, 2), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '15.048543689320388', 'number of elements evaluated': '2.06e+02'}, {'number of elements': 9475854336, 'syntax': (0, 1, 2, 3, 1, 0), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '5.848329048843188', 'number of elements evaluated': '3.11e+03'}]


...

[["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"]]
[{'number of elements': 922417564483584, 'syntax': [0, 1, 2, 3, 4, 5], 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': 0, 'number of elements evaluated': 0}, {'number of elements': 97344, 'syntax': (0, 1, 0, 1, 0, 1), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '20.0', 'number of elements evaluated': '5.00e+01'}, {'number of elements': 30371328, 'syntax': (0, 1, 2, 0, 1, 2), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '15.048543689320388', 'number of elements evaluated': '2.06e+02'}, {'number of elements': 9475854336, 'syntax': (0, 1, 2, 3, 1, 0), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '5.848329048843188', 'number of elements evaluated': '3.11e+03'}, {'number of elements': 9475854336, 'syntax': (0, 1, 2, 3, 1, 0), 'conditions': [["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (

[["'IP (eV)'#<=#7"], [], ["'IP (eV)'#<=#7"], [], ["'IP (eV)'#<=#7"], []]
[{'number of elements': 922417564483584, 'syntax': [0, 1, 2, 3, 4, 5], 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': 0, 'number of elements evaluated': 0}, {'number of elements': 97344, 'syntax': (0, 1, 0, 1, 0, 1), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '20.0', 'number of elements evaluated': '5.00e+01'}, {'number of elements': 30371328, 'syntax': (0, 1, 2, 0, 1, 2), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '15.048543689320388', 'number of elements evaluated': '2.06e+02'}, {'number of elements': 9475854336, 'syntax': (0, 1, 2, 3, 1, 0), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '5.848329048843188', 'number of elements evaluated': '3.11e+03'}, {'number of elements': 9475854336, 'syntax': (0, 1, 2, 3, 1, 0), 'conditions': [["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#

[["'IP (eV)'#<=#7"], [], ["'IP (eV)'#<=#7"], [], ["'IP (eV)'#<=#7"], []]
[{'number of elements': 922417564483584, 'syntax': [0, 1, 2, 3, 4, 5], 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': 0, 'number of elements evaluated': 0}, {'number of elements': 97344, 'syntax': (0, 1, 0, 1, 0, 1), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '20.0', 'number of elements evaluated': '5.00e+01'}, {'number of elements': 30371328, 'syntax': (0, 1, 2, 0, 1, 2), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '15.048543689320388', 'number of elements evaluated': '2.06e+02'}, {'number of elements': 9475854336, 'syntax': (0, 1, 2, 3, 1, 0), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '5.848329048843188', 'number of elements evaluated': '3.11e+03'}, {'number of elements': 9475854336, 'syntax': (0, 1, 2, 3, 1, 0), 'conditions': [["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#

[["'IP (eV)'#<=#7"], ["'IP (eV)'#>=#7.5"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#>=#7.5"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#>=#7.5"]]
[{'number of elements': 922417564483584, 'syntax': [0, 1, 2, 3, 4, 5], 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': 0, 'number of elements evaluated': 0}, {'number of elements': 97344, 'syntax': (0, 1, 0, 1, 0, 1), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '20.0', 'number of elements evaluated': '5.00e+01'}, {'number of elements': 30371328, 'syntax': (0, 1, 2, 0, 1, 2), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '15.048543689320388', 'number of elements evaluated': '2.06e+02'}, {'number of elements': 9475854336, 'syntax': (0, 1, 2, 3, 1, 0), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '5.848329048843188', 'number of elements evaluated': '3.11e+03'}, {'number of elements': 9475854336, 'syntax': (0, 1, 2, 3, 1, 0), 'conditions': [["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], [

[["'IP (eV)'#<=#7"], [], ["'IP (eV)'#<=#7"], [], ["'IP (eV)'#<=#7"], []]
[{'number of elements': 922417564483584, 'syntax': [0, 1, 2, 3, 4, 5], 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': 0, 'number of elements evaluated': 0}, {'number of elements': 97344, 'syntax': (0, 1, 0, 1, 0, 1), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '20.0', 'number of elements evaluated': '5.00e+01'}, {'number of elements': 30371328, 'syntax': (0, 1, 2, 0, 1, 2), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '15.048543689320388', 'number of elements evaluated': '2.06e+02'}, {'number of elements': 9475854336, 'syntax': (0, 1, 2, 3, 1, 0), 'conditions': [[], [], [], [], [], []], 'Elements in top 5%': '5.848329048843188', 'number of elements evaluated': '3.11e+03'}, {'number of elements': 9475854336, 'syntax': (0, 1, 2, 3, 1, 0), 'conditions': [["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#7"], ["'IP (eV)'#<=#

In [73]:
df_SP = pd.read_pickle('data/search_space_properties_2023_10_30_16_15_58.pkl')
df_SP

,number of elements,syntax,conditions,Elements in top 5%,number of elements evaluated
0,922417564483584,"[0, 1, 2, 3, 4, 5]","[[], [], [], [], [], []]",0,0
1,97344,"(0, 1, 0, 1, 0, 1)","[[], [], [], [], [], []]",20.0,5.00e+01
2,30371328,"(0, 1, 2, 0, 1, 2)","[[], [], [], [], [], []]",15.048543689320388,2.06e+02
3,9475854336,"(0, 1, 2, 3, 1, 0)","[[], [], [], [], [], []]",5.848329048843188,3.11e+03
4,9475854336,"(0, 1, 2, 3, 1, 0)","[['IP (eV)'#<=#7], ['IP (eV)'#<=#7], ['IP (eV)...",7.662082514734774,1.02e+03
5,9475854336,"(0, 1, 2, 3, 1, 0)","[['IP (eV)'#<=#7], [], ['IP (eV)'#<=#7], [], [...",8.29576194770063,1.11e+03
6,9475854336,"(0, 1, 2, 3, 1, 0)","[['IP (eV)'#<=#7], [], ['IP (eV)'#<=#7], [], [...",8.29576194770063,1.11e+03
7,9475854336,"(0, 1, 2, 3, 1, 0)","[['IP (eV)'#<=#7], ['IP (eV)'#>=#7.5], ['IP (e...",no data,0.00e+00


In [75]:
# load the saved dataframe with all the tested conditions
# and output it in order to select a few to add to the bactk experiment

#df_SP = pd.read_pickle('data/search_space_properties_2023_10_30_11_19_54.pkl')
df_SP = pd.read_pickle('data/search_space_properties_2023_10_30_16_20_54.pkl')

import pickle
from datetime import datetime
import os
def save_SP_for_EXP(df:pd.DataFrame,index_list:list,exp_name:str):
    time_now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    save_dir = f'data/input/EXP_search_sapce/{exp_name}_{time_now}/'
    os.makedirs(save_dir,exist_ok=True)
    SP = Searched_space.Searched_Space(number_of_fragments=6,df=df_precursors,features_frag=df_precursors.columns[0:1],generation_type='conditional')
    SP.features_frag=SP.features_frag.append(df_precursors.columns[17:23])
    SP.features_frag=SP.features_frag.append(df_precursors.columns[1:7])

    for id in index_list:
        SP.conditions_list = df['conditions'].values[id]
        SP.syntax = list(df['syntax'].values[id])
        #SP.redefine_search_space()
        print(SP.space_size)
        print(SP.syntax)
        print(SP.conditions_list)
        pickle.dump(SP,file=open(save_dir+f'/search_space_{id}.pkl','wb'))
    return SP


df_SP


,number of elements,syntax,conditions,Elements in top 5%,number of elements evaluated
0,922417564483584,"[0, 1, 2, 3, 4, 5]","[[], [], [], [], [], []]",0,0
1,97344,"(0, 1, 0, 1, 0, 1)","[[], [], [], [], [], []]",20.0,5.00e+01
2,30371328,"(0, 1, 2, 0, 1, 2)","[[], [], [], [], [], []]",15.048543689320388,2.06e+02
3,9475854336,"(0, 1, 2, 3, 1, 0)","[[], [], [], [], [], []]",5.848329048843188,3.11e+03
4,9475854336,"(0, 1, 2, 3, 1, 0)","[['IP (eV)'#<=#7], ['IP (eV)'#<=#7], ['IP (eV)...",7.662082514734774,1.02e+03
5,9475854336,"(0, 1, 2, 3, 1, 0)","[['IP (eV)'#<=#7], [], ['IP (eV)'#<=#7], [], [...",8.29576194770063,1.11e+03
6,9475854336,"(0, 1, 2, 3, 1, 0)","[['IP (eV)'#<=#7], [], ['IP (eV)'#<=#7], [], [...",8.29576194770063,1.11e+03
7,9475854336,"(0, 1, 2, 3, 1, 0)","[['IP (eV)'#<=#7], ['IP (eV)'#>=#7.5], ['IP (e...",no data,0.00e+00
8,922417564483584,"(0, 1, 2, 3, 4, 5)","[['IP (eV)'#<=#7], [], ['IP (eV)'#<=#7], [], [...",7.498601007274762,1.25e+04


In [76]:
SP= save_SP_for_EXP(df_SP,[0,3,5,8],'exp_111')


922417564483584
[0, 1, 2, 3, 4, 5]
[[], [], [], [], [], []]
922417564483584
[0, 1, 2, 3, 1, 0]
[[], [], [], [], [], []]
922417564483584
[0, 1, 2, 3, 1, 0]
[["'IP (eV)'#<=#7"], [], ["'IP (eV)'#<=#7"], [], ["'IP (eV)'#<=#7"], []]
922417564483584
[0, 1, 2, 3, 4, 5]
[["'IP (eV)'#<=#7"], [], ["'IP (eV)'#<=#7"], [], ["'IP (eV)'#<=#7"], []]


In [46]:

def get_prob(df:pd.DataFrame,index_list:list,exp_name:str,target_limit=-3):
    time_now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    save_dir = f'data/input/EXP_search_sapce/{exp_name}_{time_now}/'
    os.makedirs(save_dir,exist_ok=True)
    SP = Searched_space.Searched_Space(number_of_fragments=6,df=df_precursors,features_frag=df_precursors.columns[0:1],generation_type='conditional')
    SP.features_frag=SP.features_frag.append(df_precursors.columns[17:23])
    SP.features_frag=SP.features_frag.append(df_precursors.columns[1:7])
    success_rate,num_evaluation = [],[]
    for id in index_list:
        SP.conditions_list = df['conditions'].values[id]
        SP.syntax = list(df['syntax'].values[id])
        df_SP= SP.check_df_for_element_from_SP(df_to_check=df_total)
        print(id,df_SP[df_SP['target']>target_limit].shape[0]/df_SP.shape[0])
        success_rate.append( df_SP[df_SP['target']>target_limit].shape[0]/df_SP.shape[0])
        num_evaluation.append(df_SP.shape[0])
        #SP.redefine_search_space()
        #print(SP.space_size)
        #print(SP.syntax)
        #print(SP.conditions_list)
        #pickle.dump(SP,file=open(save_dir+f'/search_space_{id}.pkl','wb'))
    df_res = pd.DataFrame({'id':index_list,'success_rate':success_rate,'num_evaluation':num_evaluation}).T
    return df_res
df_res= get_prob(df_SP,[0,1,2,3,4,5,6,7],'exp_test',target_limit=-3)
df_res

0 0.0027979854504756574
1 0.0
2 0.0
3 0.03333333333333333
4 0.03333333333333333
5 0.005488474204171241
6 0.005488474204171241
7 0.005488474204171241


,0,1,2,3,4,5,6,7
id,0.000000,1.0,2.0,3.000000,4.000000,5.000000,6.000000,7.000000
success_rate,0.002798,0.0,0.0,0.033333,0.033333,0.005488,0.005488,0.005488
num_evaluation,12509.000000,5.0,37.0,60.000000,60.000000,911.000000,911.000000,911.000000


In [ ]:
pickle.load(open(self.search_space_loc, "rb"))

In [15]:
def plot_histogram_fragment(df,df_total,number_of_fragments):
        def plot_histogram_fragment(column_name,df,df_total,number_of_fragments):
            fig, ax = plt.subplots(3, 2, figsize=(6, 6),sharex='col', sharey='row')
            # ax share x axis and y axis
            #ax = ax.flatten()

            for i in range(number_of_fragments):
                ax[i // 2, i % 2].hist(
                    df_total[f"{column_name}_{i}"],
                    bins=20,
                    edgecolor="black",
                    density=True,
                    label = "all"
                )
                ax[i // 2, i % 2].hist(
                    df[f"{column_name}_{i}"],
                    bins=20,
                    edgecolor="black",
                    density=True,
                    label = "current"
                )
                ax[i // 2, i % 2].set_xlabel(f"{column_name}_{i}")
                ax[i // 2, i % 2].legend()
                #ax[i // 2, i % 2].set_ylim([0, 1])
                #ax[i // 2, i % 2].set_ylabel("Frequency")
                #ax[i // 3, i % 3].set_title(f"Histogram of {column_name}_{i}")
            # set xlabel and y label for the last row

            plt.tight_layout()
            plt.show()

        # Interactive widget for column selection
        columns_dropdown = widgets.Dropdown(
            options=[x.replace('_0','') for x in df.select_dtypes(include=["int", "float"]).columns if '_0' in x],
            description="Value:",
        )
        # Set up the layout of the widgets

        vbox_layout = Layout(display="flex", flex_flow="row", align_items="flex-start")
        # Display the widget
        interact(plot_histogram, column_name=columns_dropdown, layout=vbox_layout)
        plt.show()
SP.plot_histogram_precursor()

interactive(children=(Dropdown(description='Value:', options=('Atom_num', 'HOMO (eV)', 'LUMO (eV)', 'HOMO_LUMO…

In [20]:
def plot_histogram_2(column_name,df_current_2):
    fig, ax = plt.subplots(3, 2, figsize=(6, 6),sharex='col', sharey='row')
    for i in range(SP.number_of_fragments):
        ax[i // 2, i % 2].hist(
            df_total[f"{column_name}_{i}"],
            bins=20,
            edgecolor="black",
            density=True,
            label = "all"
        )
        ax[i // 2, i % 2].hist(
            df_current_2[f"{column_name}_{i}"],
            bins=20,
            edgecolor="black",
            density=True,
            label = "current"
        )
        ax[i // 2, i % 2].set_xlabel(f"{column_name}_{i}")
        ax[i // 2, i % 2].legend()
    # set xlabel and y label for the last row

    plt.tight_layout()
    plt.show()
#df_total=df_test
df= SP.check_df_for_element_from_SP(df_to_check=df_total)
# Interactive widget for column selection
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from matplotlib import pyplot as plt
columns_dropdown_2 = widgets.Dropdown(
    options=[x.replace('_0','') for x in df_total.select_dtypes(include=["int", "float"]).columns if '_0' in x],
    description="Value:",
)
hist_widget = interactive(plot_histogram_2, column_name=widgets.fixed('Atom_num'),df_current_2=widgets.fixed(df))
display(hist_widget)
#wait for the user to select the syntax
input("Press Enter to continue...")
hist_widget.kwargs['df_current_2']=SP.check_df_for_element_from_SP(df_to_check=df_total)[:5]

hist_widget.update()

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [11]:
list_col = [f'InChIKey_{x}' for x in range(6)]
list_col.append('target')
df_total[list_col].to_csv('Data/output/Full_datatset/data_for_adam.csv',index=False)